In [8]:
from datetime import datetime
datetime.strptime('15/06/2022', "%d/%m/%Y").strftime("%y-%M-%D")

'06/15/22'

In [10]:
!pip install chrombadb

ERROR: Could not find a version that satisfies the requirement chrombadb (from versions: none)
ERROR: No matching distribution found for chrombadb


In [15]:
from langchain.vectorstores import Chroma 
vectorstore = Chroma(collection_name="langchain_store", client_settings={})

In [14]:
config("ADMIN")

'admin'

In [ ]:
chroma_server_host: Optional[str] = None
    chroma_server_headers: Optional[Dict[str, str]] = None
    chroma_server_http_port: Optional[str] = None
    chroma_server_ssl_enabled: Optional[bool] = False
    chroma_server_grpc_port: Optional[str] = None
    chroma_server_cors_allow_origins: List[str] = []  #

In [8]:
from base64 import b64encode

# Encode the username and password for Basic Authentication
credentials = b64encode("admin:admin".encode()).decode("utf-8")
headers = {
    "Authorization": f"Basic {credentials}"
}
headers

{'Authorization': 'Basic YWRtaW46YWRtaW4='}

In [5]:
from decouple import config
from base64 import b64encode
logs = "admin:{}".format(config("ADMIN"))
logs_byes = logs.encode("utf-8")
encoded_logs = b64encode(logs_byes)
encoded_logs.decode("utf-8")

'YWRtaW46YWRtaW4='

In [13]:
import chromadb
from chromadb.config import Settings
from decouple import config
chroma_client = chromadb.HttpClient(settings=Settings(allow_reset=True,
 ), host = "localhost", port = 8001, headers=headers)
chroma_client.reset()


True

In [10]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
chromadb = Chroma(client = chroma_client, collection_name = "smart_maya", embedding_function=OpenAIEmbeddings())

In [11]:
chromadb.similarity_search("Impot")

[]

In [ ]:
# create the chroma client
import chromadb
import uuid
from chromadb.config import Settings

client = chromadb.HttpClient(settings=Settings(allow_reset=True))
client.reset()  # resets the database
collection = client.create_collection("my_collection")
for doc in docs:
    collection.add(
        ids=[str(uuid.uuid1())], metadatas=doc.metadata, documents=doc.page_content
    )

# tell LangChain to use our client and collection name
db4 = Chroma(client=client, collection_name="my_collection", embedding_function=embedding_function)
query = "What did the president say about Ketanji Brown Jackson"
docs = db4.similarity_search(query)
print(docs[0].page_content)

In [17]:
import chromadb
import uuid
from chromadb.config import Settings

In [19]:
client = chromadb.HttpClient(settings=Settings(allow_reset=True))
client.reset() 

Exception: {"detail":"Not Found"}